In [6]:
from gmm_denoiser import GMMDenoiser
from regularized_denoisers import denoise_l2
from skimage.restoration import denoise_wavelet
from utils import *
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [7]:
df = pd.read_excel("data/PD data.xlsx")

In [8]:
train_signals = [
    np.array(df["s1sensor1"]),
    np.array(df["s1sensor2"]),
    np.array(df["s1sensor3"]),
    np.array(df["s1sensor4"]),
    np.array(df["s1sensor5"]),
]

In [9]:
denoiser = GMMDenoiser(gmm_n_components=10, patch_size=30, train_signals=train_signals)
denoiser.fit()

In [10]:
import keras
model = keras.Sequential([
    keras.layers.Input(shape=(2500, 1)),
    keras.layers.Conv1D(filters=32, kernel_size=9, activation="relu", padding="same"),
    keras.layers.MaxPooling1D(2),
    keras.layers.Conv1D(filters=32, kernel_size=9, activation="relu", padding="same"),
    keras.layers.MaxPooling1D(2),

    keras.layers.Conv1DTranspose(filters=32, kernel_size=9, strides=2, activation="relu", padding="same"),
    keras.layers.Conv1DTranspose(filters=32, kernel_size=9, strides=2, activation="relu", padding="same"),
    keras.layers.Conv1D(1, 9, padding="same")
])
model.compile(optimizer="adam", loss="mse")

model_X = []; model_y = []
for ts in train_signals:
    model_X.append(ts)
    model_y.append(ts)
    for snr in [-10, -5, 0, 5, 10]:
      model_X.append(awgn(ts, snr))
      model_y.append(ts)
model_X = np.array(model_X).reshape(30, 2500, 1)
model_y = np.array(model_y).reshape(30, 2500, 1)
model.fit(model_X, model_y, batch_size=1, epochs=10)

2023-09-07 03:42:16.375037: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-07 03:42:18.858879: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-07 03:42:18.859933: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-07 03:42:19.941545: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Epoch 1/10
24/30 [=======================>......] - ETA: 0s - loss: 0.0659

2023-09-07 03:42:23.451088: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31680000 exceeds 10% of free system memory.
2023-09-07 03:42:23.452953: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28800000 exceeds 10% of free system memory.
2023-09-07 03:42:23.454557: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 27360000 exceeds 10% of free system memory.
2023-09-07 03:42:23.455326: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 28800000 exceeds 10% of free system memory.
2023-09-07 03:42:23.462592: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 31680000 exceeds 10% of free system memory.


30/30 [==============================] - 1s 7ms/step - loss: 0.0610
Epoch 2/10
30/30 [==============================] - 0s 7ms/step - loss: 0.0273
Epoch 3/10
30/30 [==============================] - 0s 13ms/step - loss: 0.0164
Epoch 4/10
30/30 [==============================] - 0s 6ms/step - loss: 0.0115
Epoch 5/10
30/30 [==============================] - 0s 8ms/step - loss: 0.0107
Epoch 6/10
30/30 [==============================] - 0s 7ms/step - loss: 0.0108
Epoch 7/10
30/30 [==============================] - 0s 6ms/step - loss: 0.0098
Epoch 8/10
30/30 [==============================] - 0s 6ms/step - loss: 0.0099
Epoch 9/10
30/30 [==============================] - 0s 6ms/step - loss: 0.0099
Epoch 10/10
30/30 [==============================] - 0s 9ms/step - loss: 0.0104


In [11]:
test_signals = [
    np.array(df["s2sensor1"]),
    np.array(df["s2sensor2"]),
    np.array(df["s2sensor3"]),
    np.array(df["s2sensor4"]),
    np.array(df["s2sensor5"]),
]

In [12]:
test_snrs = [-10, -5, 0, 5, 10]
denoisers = ["none", "wavelet", "dae", "gmm"]
results = np.zeros((len(test_signals), len(test_snrs), len(denoisers), len(test_signals[0])))

In [13]:
for i, nf in enumerate(test_signals):
    for j, snr in enumerate(test_snrs):
        print(i, j)
        ns = awgn(nf, snr)
        results[i, j, 0, :] = ns
        results[i, j, 1, :] = denoise_wavelet(ns, method="BayesShrink", mode="soft", wavelet_levels=3, wavelet="sym8", rescale_sigma="True")
        results[i, j, 2, :] = np.array(model(ns.reshape(2500, 1).reshape(1, -1))).reshape((2500,))
        results[i, j, 3, :] = denoiser.denoise(ns, snr_db_est=snr)


0 0


0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4
4 0
4 1
4 2
4 3
4 4


In [14]:
print(f"{'method':^15}|{'snr':^8}|{'psnr':^8}|{'mse':^8}|{'ssim':^8}|{'tv':^8}")
csv_s = f"{'method':^15}|{'snr':^8}|{'psnr':^8}|{'mse':^8}|{'ssim':^8}|{'tv':^8}".replace("|",",").replace(" ", "")
for i, nf in enumerate(test_signals):
    for j, snr in enumerate(test_snrs):
        ns = results[i, j, 0]
        for name, dns in zip(denoisers, results[i, j]):
            print(f"{name:<15}|{calc_snr(nf, dns):>8.2f}|{psnr(nf, dns):>8.2f}|{mse(nf, dns):>8.4f}|{ssim(nf, dns):>8.4f}|{tv(dns):>8.1f}")
            csv_s += f"\n{name:<15}|{calc_snr(nf, dns):>8.2f}|{psnr(nf, dns):>8.2f}|{mse(nf, dns):>8.4f}|{ssim(nf, dns):>8.4f}|{tv(dns):>8.1f}".replace("|",",").replace(" ", "")


    method     |  snr   |  psnr  |  mse   |  ssim  |   tv   
none           |   -9.80|    2.46|  3.2754|  0.0036|  5085.3
wavelet        |   -0.85|   11.41|  0.4172|  0.1916|   296.6
dae            |    5.17|   17.43|  0.1044|  0.4289|   111.5
gmm            |    4.06|   16.32|  0.1348|  0.3784|    31.2
none           |   -5.06|    7.20|  1.1001|  0.0151|  2952.5
wavelet        |    4.23|   16.49|  0.1296|  0.3090|   180.9
dae            |    8.96|   21.22|  0.0436|  0.6408|    78.0
gmm            |    8.24|   20.50|  0.0515|  0.6266|    41.6
none           |    0.01|   12.27|  0.3425|  0.0600|  1643.3
wavelet        |    9.06|   21.32|  0.0426|  0.5616|   112.7
dae            |   13.36|   25.62|  0.0158|  0.8168|    66.0
gmm            |   13.46|   25.72|  0.0155|  0.8157|    50.7
none           |    5.02|   17.28|  0.1080|  0.1751|   939.0
wavelet        |   14.46|   26.72|  0.0123|  0.7829|    73.1
dae            |   14.82|   27.08|  0.0113|  0.8685|    58.4
gmm            |   17.36

In [16]:
plt.ioff()
import pathlib
pathlib.Path("out/pd_out/persigsnr").mkdir(exist_ok=True, parents=True)
for i, nf in enumerate(test_signals):
    for j, snr in enumerate(test_snrs):
        print(i, j)
        fig, axs = plt.subplots(1, 4, figsize=(12, 3))
        axs[0].set_title("Noisy")
        axs[1].set_title("Wavelet Denoised")
        axs[2].set_title("DAE Denoised")
        axs[3].set_title("GMM Denoised")
        for k, dns in enumerate(results[i, j]):
            axs[k].plot(dns, linewidth=0.5, label=("noisy" if k == 0 else "denoised"))
            axs[k].plot(nf, linewidth=0.5, label="clean")
            axs[k].text(0.5, 0.8, f"snr = {calc_snr(nf, dns):.2f}", transform=axs[k].transAxes)
            axs[k].legend(loc="lower right")
        fig.savefig(f"out/pd_out/persigsnr/sig{i}snr{snr}.png")
plt.ion()


0 0


0 1
0 2
0 3
0 4
1 0
1 1
1 2
1 3
1 4
2 0
2 1
2 2
2 3
2 4
3 0
3 1
3 2
3 3
3 4


/tmp/ipykernel_7332/273714997.py:7: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.
  fig, axs = plt.subplots(1, 4, figsize=(12, 3))


4 0
4 1
4 2
4 3
4 4
